# Exploring SkySat video for DEM generation

This dataset was acquired over Breckenridge Colorado on 2018-11-07



In [ ]:
# Analysis packages
import geopandas as gpd
import gcsfs
from shapely import wkt
import rasterio
import numpy as np

# Visualization packages
import matplotlib.pyplot as plt
from IPython.display import HTML
import geoviews as gv
import holoviews as hv
hv.notebook_extension('bokeh')
%matplotlib inline

### We've stored the data in a GCS bucket. list the contents

In [ ]:
bucket = 'skysat'

fs = gcsfs.GCSFileSystem(project='pangeo-181919')
objects = fs.ls(f'pangeo-data/{bucket}')

print('Number of objects:', len(objects))
print('Objects:', objects)

In [ ]:
fs.ls('pangeo-data/skysat/breckenridge/video/unstabilized/unstabilized.mp4')

### Look at the unstabilized video to get a feel for the dataset

* Notice that the imagery is acquired at an oblique angle (40 to 60 degrees off nadir)

In [ ]:
remotePath = 'pangeo-data/skysat/breckenridge/video/unstabilized/unstabilized.mp4'
fs.get(remotePath, './unstabilized.mp4')

In [ ]:
%%HTML
<video width="600" height="400" controls>
  <source src="./unstabilized.mp4" type="video/mp4">
</video>

### metadata.txt

In [ ]:
# NOTE: had to enclose polygon in quotes in order to load
#pangeo-data/skysat/breckenridge/video/metadata.txt
path = 'pangeo-data/skysat/breckenridge/video/metadata.txt'
with fs.open(path) as f:
    df = gpd.pd.read_csv(f, header=None, names=['key', 'value'], quotechar='"', index_col='key').T

In [ ]:
print(list(df.keys()))

In [ ]:
df.SunAzimuth

In [ ]:
df.SunElevation

In [ ]:
# Put the polygon on a map to familiarize with study area
# NOTE: this is a slippy map! You can zoom out and pan around
footprint = gv.Shape(wkt.loads(df.ExtentsPolygon.value), label='Skysat footprint').options(alpha=0.5)
tiles = gv.tile_sources.StamenTerrainRetina(width=700, height=500)
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprint * labels

### frame_index.csv

In [ ]:
def fix_polygon_wkt(string):
    '''returns shapely geometry from reformatted WKT'''
    pre = string[:-2]
    first_point = string.split(',')[0].split('(')[-1]
    fixed = f'{pre},{first_point}))'
    return wkt.loads(fixed)

In [ ]:
# Load frames GeoDataFrame
with fs.open('pangeo-data/skysat/breckenridge/video/frame_index.csv') as f:
    tmp = gpd.pd.read_csv(f)

# NOTE: timestamps not best for index since precision to second has repeat values
#tmp = tmp.set_index(gpd.pd.to_datetime(tmp['datetime']))
tmp['datetime'] = gpd.pd.to_datetime(tmp.datetime)
tmp['geom'] = tmp.geom.apply(fix_polygon_wkt)
tmp['path'] = '/pangeo-data/skysat/breckenridge/video/frames/' + tmp['name'] + '.tiff'

gf = gpd.GeoDataFrame(tmp, crs = {'init': 'epsg:4326'}, geometry='geom')

In [ ]:
gf.head()

In [ ]:
gf.geometry.plot() 

In [ ]:
print('UTC TIME:')
print('Start:', gf.datetime.iloc[0])
print('End: ', gf.datetime.iloc[-1])
# NOTE: Colorado is -7 comparted to UTC
print('Local Time ~ 10:50 AM')

In [ ]:
gf.loc[:,('sat_az','sat_elev')].describe()

In [ ]:
# Plot how satellite az and elevation change in time
#plt.plot(gf.datetime, gf.sat_az)
gf.sat_az.plot()
for frame in [0,449,898,1348]:
    plt.axvline(frame, linestyle=':')
plt.ylabel('Satellite Azimuth')
plt.title('SkySat Video - Breckenridge 2018-11-07');
plt.xlabel('Frame #');
plt.savefig('sat_az.pdf', bbox_inches='tight')

In [ ]:
# Note: how different are subsequent time steps?
print(gf.datetime.iloc[1], gf.datetime.iloc[2])
# only precision to nearest second, therefore 'stairstep ' appearance

In [ ]:
gf.sat_elev.plot()
for frame in [0,449,898,1348]:
    plt.axvline(frame, linestyle=':')
plt.ylabel('Satellite Elevation')
plt.xlabel('Frame #')
plt.title('SkySat Video - Breckenridge 2018-11-07');
plt.savefig('sat_elev.pdf', bbox_inches='tight')

In [ ]:
# Recorded satellite position in time
df = gpd.pd.DataFrame(gf.loc[:, ('q0', 'q1', 'q2', 'q3')])
df.plot()

In [ ]:
# Recorded satellite position in time
df = gpd.pd.DataFrame(gf.loc[:, ('x_sat_eci', 'y_sat_eci', 'z_sat_eci')])
df.plot()

In [ ]:
# Pairs chosen for use in example workflow
v1 = '1225648254.44006968_sc00004_c1_PAN'
v2 = '1225648269.40892076_sc00004_c1_PAN'
v3 = '1225648284.37777185_sc00004_c1_PAN'
v4 = '1225648299.37995577_sc00004_c1_PAN'

subset = [v1,v2,v3,v4]

In [ ]:
sub = gf[gf.name.isin(subset)]

In [ ]:
sub

# visualize a single frame

In [ ]:
sub.path.iloc[0]

In [ ]:
# try connecting directly to bucket w/ rasterio
gsPath = 'gs://pangeo-data/skysat/breckenridge/video/frames/1225648254.44006968_sc00004_c1_PAN.tiff'
with rasterio.open(gsPath) as src:
    print(src.profile)
    data = src.read(1)

In [ ]:
# Or access via gcsfs...
with fs.open(sub.path.iloc[0]) as f:
    with rasterio.open(f) as src:
        print(src.profile)
        data = src.read(1)

In [ ]:
plt.imshow(data, cmap='gray')